# 使用三种API开放平台实现人脸识别

* 产品设计：在某些活动里使用人脸识别，当天已购票的人可以通过人脸识别重复多次进入会场。
* [示例图片](https://c-ssl.duitang.com/uploads/item/202003/05/20200305113817_bplom.thumb.1000_0.jpg)

## Acure

In [4]:
import requests
import json

KEY = 'f701f8dd6fc1483fb8aed386a75b9b8e'  # Replace with a valid Subscription Key here.
BASE_URL = 'https://eastasia.api.cognitive.microsoft.com/face/v1.0/detect' 

# 沿用API的示范代碼，{subscription key}用KEY代入
HEADERS = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': '{}'.format(KEY),
}

img_url = 'https://c-ssl.duitang.com/uploads/item/202003/05/20200305113817_bplom.thumb.1000_0.jpg'
data = {
    'url': '{}'.format(img_url),
}
payload = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'flase',
    'returnFaceAttributes': '{}'.format('age,gender,glasses'), 
}
r = requests.post(BASE_URL, data=json.dumps(data), params=payload, headers=HEADERS)

r.status_code
r.json()

[{'faceId': '7d4a8360-3200-4153-90ea-b3cf7fd17405',
  'faceRectangle': {'top': 717, 'left': 605, 'width': 253, 'height': 253},
  'faceAttributes': {'gender': 'male', 'age': 19.0, 'glasses': 'NoGlasses'}},
 {'faceId': '54c54216-596e-4f5a-a87a-b6b17c43b103',
  'faceRectangle': {'top': 709, 'left': 128, 'width': 248, 'height': 248},
  'faceAttributes': {'gender': 'male', 'age': 23.0, 'glasses': 'NoGlasses'}}]

## Face++

In [5]:
import requests


api_secret = "GOIWWYgByZ_odq7kgchowa6nCZzNKVMU"
api_key = 'D-09_qb6yShR-K3ydXgs0ajfEf1SZhcX'  # Replace with a valid Subscription Key here.

BASE_URL = 'https://api-cn.faceplusplus.com/facepp/v3/detect' 
img_url = 'https://c-ssl.duitang.com/uploads/item/202003/05/20200305113817_bplom.thumb.1000_0.jpg'

headers = {
    'Content-Type': 'application/json',
}

payload = {
    "image_url":img_url,
    'api_key': api_key,
    'api_secret': api_secret,
    'return_attributes':'gender,age,smiling,emotion', 
}

r = requests.post(BASE_URL, params=payload, headers=headers)

r.status_code
print(r.content)

b'{"request_id":"1586496042,4dc64124-446a-46a8-96e1-e73ac433f1ec","time_used":620,"faces":[{"face_token":"fe850433008412dc0a6da0600ef7af2b","face_rectangle":{"top":724,"left":121,"width":259,"height":259},"attributes":{"gender":{"value":"Male"},"age":{"value":23},"smile":{"value":4.485,"threshold":50.000},"emotion":{"anger":0.040,"disgust":0.022,"fear":0.028,"happiness":1.483,"neutral":1.230,"sadness":0.022,"surprise":97.174}}},{"face_token":"b1e5ff346f94f9390dc664943ea90b6e","face_rectangle":{"top":738,"left":606,"width":238,"height":238},"attributes":{"gender":{"value":"Male"},"age":{"value":23},"smile":{"value":0.064,"threshold":50.000},"emotion":{"anger":2.139,"disgust":0.025,"fear":0.025,"happiness":0.617,"neutral":89.831,"sadness":7.337,"surprise":0.025}}}],"image_id":"toNPEzcrZncC0G+vHab8SQ==","face_num":2}\n'


## 腾讯AI

In [6]:
import hashlib  
import time  
import random  
import string
import requests  
import base64  
import requests

import numpy as np
from urllib.parse import urlencode
import json #用于post后得到的字符串到字典的转换

app_id = '2131725665' 
app_key = 'OlsChRvRR5tCUgcd'

'''
        腾讯openai鉴权签名计算步骤：（摘抄自官网）
            用于计算签名的参数在不同接口之间会有差异，但算法过程固定如下4个步骤。
        1 将<key, value>请求参数对按key进行字典升序排序，得到有序的参数对列表N
        2 将列表N中的参数对按URL键值对的格式拼接成字符串，得到字符串T（如：key1=value1&key2=value2），URL键值拼接过程value部分需要URL编码，URL编码算法用大写字母，例如%E8，而不是小写%e8
        3 将应用密钥以app_key为键名，组成URL键值拼接到字符串T末尾，得到字符串S（如：key1=value1&key2=value2&app_key=密钥)
        4 对字符串S进行MD5运算，将得到的MD5值所有字符转换成大写，得到接口请求签名
'''

def face_check(img_data):
    """
    人脸识别demo
    :param img_data: 二进制的图片数据
    :return:
    """

    data = base64.b64encode(img_data)

    image = data.decode()

    imageType = "BASE64"
    
def get_params(img):                         #鉴权计算并返回请求参数
    #请求时间戳（秒级），用于防止请求重放（保证签名5分钟有效
    time_stamp=str(int(time.time())) 
    #请求随机字符串，用于保证签名不可预测,16代表16位
    nonce_str = ''.join(random.sample(string.ascii_letters + string.digits, 16))

    params = {'app_id':app_id,                #请求包，需要根据不同的任务修改，基本相同
              'image':img,                    #文字类的任务可能是‘text’，由主函数传递进来
              'mode':'0' ,                    #身份证件类可能是'card_type'
              'time_stamp':time_stamp,        #时间戳，都一样
              'nonce_str':nonce_str,          #随机字符串，都一样
              #'sign':''                      #签名不参与鉴权计算，只是列出来示意
             }

    sort_dict= sorted(params.items(), key=lambda item:item[0], reverse = False)  #字典排序
    sort_dict.append(('app_key',app_key))   #尾部添加appkey
    rawtext= urlencode(sort_dict).encode()  #urlencod编码
    sha = hashlib.md5()    
    sha.update(rawtext)
    md5text= sha.hexdigest().upper()        #MD5加密计算
    params['sign']=md5text                  #将签名赋值到sign
    return  params                          #返回请求包

with open(r"C:\Users\ASUS\prem.jpg","rb")as f:
    base64_data = base64.b64encode(f.read())
    
params = get_params(base64_data)

url = "https://api.ai.qq.com/fcgi-bin/face/face_detectface"
res = requests.post(url,params).json()
res

{'ret': 0,
 'msg': 'ok',
 'data': {'image_width': 944,
  'image_height': 1893,
  'face_list': [{'face_id': '3588446062641190045',
    'x': 588,
    'y': 666,
    'width': 292,
    'height': 292,
    'gender': 99,
    'age': 24,
    'expression': 0,
    'beauty': 77,
    'glass': 0,
    'pitch': 4,
    'yaw': 9,
    'roll': -3,
    'face_shape': {'face_profile': [{'x': 614, 'y': 754},
      {'x': 612, 'y': 780},
      {'x': 613, 'y': 805},
      {'x': 617, 'y': 831},
      {'x': 622, 'y': 856},
      {'x': 631, 'y': 880},
      {'x': 644, 'y': 902},
      {'x': 660, 'y': 922},
      {'x': 678, 'y': 940},
      {'x': 700, 'y': 953},
      {'x': 725, 'y': 957},
      {'x': 748, 'y': 952},
      {'x': 768, 'y': 938},
      {'x': 786, 'y': 921},
      {'x': 801, 'y': 902},
      {'x': 814, 'y': 881},
      {'x': 824, 'y': 859},
      {'x': 832, 'y': 836},
      {'x': 837, 'y': 813},
      {'x': 840, 'y': 788},
      {'x': 841, 'y': 767}],
     'left_eye': [{'x': 659, 'y': 758},
      {'x': 

## 运用Face++分析人脸数据制作数据表格

In [7]:
results = r.json()
results

{'request_id': '1586496042,4dc64124-446a-46a8-96e1-e73ac433f1ec',
 'time_used': 620,
 'faces': [{'face_token': 'fe850433008412dc0a6da0600ef7af2b',
   'face_rectangle': {'top': 724, 'left': 121, 'width': 259, 'height': 259},
   'attributes': {'gender': {'value': 'Male'},
    'age': {'value': 23},
    'smile': {'value': 4.485, 'threshold': 50.0},
    'emotion': {'anger': 0.04,
     'disgust': 0.022,
     'fear': 0.028,
     'happiness': 1.483,
     'neutral': 1.23,
     'sadness': 0.022,
     'surprise': 97.174}}},
  {'face_token': 'b1e5ff346f94f9390dc664943ea90b6e',
   'face_rectangle': {'top': 738, 'left': 606, 'width': 238, 'height': 238},
   'attributes': {'gender': {'value': 'Male'},
    'age': {'value': 23},
    'smile': {'value': 0.064, 'threshold': 50.0},
    'emotion': {'anger': 2.139,
     'disgust': 0.025,
     'fear': 0.025,
     'happiness': 0.617,
     'neutral': 89.831,
     'sadness': 7.337,
     'surprise': 0.025}}}],
 'image_id': 'toNPEzcrZncC0G+vHab8SQ==',
 'face_num':

In [8]:
import pandas as pd
df = pd.json_normalize(results,record_path='faces')
df = df.rename (columns = {"attributes.gender.value":"性别",
                           "attributes.age.value":"年龄",
                           "attributes.smile.value":"笑容",
                           "attributes.smile.threshold":"是否有笑容",
                           "attributes.emotion.anger":"愤怒",
                           "attributes.emotion.disgust":"厌恶",
                           "attributes.emotion.fear":"恐惧",
                           "attributes.emotion.happiness":"高兴",
                           "attributes.emotion.neutral":"平静",
                           "attributes.emotion.sadness":"悲伤",
                           "attributes.emotion.surprise":"惊讶"})
df = df.set_index('face_token')
df = df.iloc[:,4:]
df.replace({"Male":"男",
            "Female":"女",})

,性别,年龄,笑容,是否有笑容,愤怒,厌恶,恐惧,高兴,平静,悲伤,惊讶
face_token,,,,,,,,,,,
fe850433008412dc0a6da0600ef7af2b,男,23,4.485,50.0,0.040,0.022,0.028,1.483,1.230,0.022,97.174
b1e5ff346f94f9390dc664943ea90b6e,男,23,0.064,50.0,2.139,0.025,0.025,0.617,89.831,7.337,0.025
